In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
data = pd.read_csv('../input/kmeans-data/data.csv')
labels = pd.read_csv('../input/kmeans-data/label.csv',names=['label'],header=None)

In [4]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [6]:
#Split data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [7]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            # np.sum()
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict
    

In [8]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [9]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [10]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [11]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  25497.262135922327
Tolerance for the Iteration  1 :  6471.334777308546
Tolerance for the Iteration  2 :  2621.579021352727
Tolerance for the Iteration  3 :  1948.0277976025636
Tolerance for the Iteration  4 :  1580.843946716233
Tolerance for the Iteration  5 :  1716.9515404040403
Tolerance for the Iteration  6 :  1628.1503981163842
Tolerance for the Iteration  7 :  1187.4169000104162
Tolerance for the Iteration  8 :  1150.8833328577919
Tolerance for the Iteration  9 :  993.645855151008
Tolerance for the Iteration  10 :  1014.5299289157941
Tolerance for the Iteration  11 :  624.8886519378989
Tolerance for the Iteration  12 :  509.40155581344726
Tolerance for the Iteration  13 :  472.2936655244623
Tolerance for the Iteration  14 :  471.96531743765445
Tolerance for the Iteration  15 :  486.01126170395275
Tolerance for the Iteration  16 :  511.04472735747123
Tolerance for the Iteration  17 :  454.29234259317764
Tolerance for the Iteration  18 :  222.189080

In [12]:
Euclidean_SSE = model1.calculate_SSE(centroids1,clusters1,data)

In [13]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25490189976.302994


In [14]:
cluster_labels1 = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels1

array([2, 6, 1, 5, 2, 0, 9, 3, 4, 7])

In [15]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.0925

In [17]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = model2.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  41188.847272727275
Tolerance for the Iteration  1 :  12629.654736565499
Tolerance for the Iteration  2 :  9598.063043974991
Tolerance for the Iteration  3 :  1532.7810008369675
Tolerance for the Iteration  4 :  2390.325408755274
Tolerance for the Iteration  5 :  1187.3032325258957
Tolerance for the Iteration  6 :  846.967932614157
Tolerance for the Iteration  7 :  1183.1960191830703
Tolerance for the Iteration  8 :  1250.169992109343
Tolerance for the Iteration  9 :  1349.5857458352612
Tolerance for the Iteration  10 :  357.94857816674767
Tolerance for the Iteration  11 :  284.2244002302642
Tolerance for the Iteration  12 :  927.9683254907769
Tolerance for the Iteration  13 :  218.33851376415566
Tolerance for the Iteration  14 :  216.10245253338897
Tolerance for the Iteration  15 :  0.0


In [18]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [19]:
cluster_labels2 = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 8, 4, 2, 1, 3, 4, 0, 6, 6])

In [21]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [22]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  27883.698854337155
Tolerance for the Iteration  1 :  4959.417573405411
Tolerance for the Iteration  2 :  3419.5498236331573
Tolerance for the Iteration  3 :  2247.924827245488
Tolerance for the Iteration  4 :  1724.1881517606703
Tolerance for the Iteration  5 :  1571.8870160572285
Tolerance for the Iteration  6 :  1188.9197908636265
Tolerance for the Iteration  7 :  1178.6514410976697
Tolerance for the Iteration  8 :  1320.6947290981475
Tolerance for the Iteration  9 :  1084.9501699007424
Tolerance for the Iteration  10 :  980.4739112758534
Tolerance for the Iteration  11 :  919.291664531338
Tolerance for the Iteration  12 :  1126.3569405125513
Tolerance for the Iteration  13 :  945.7184420073662
Tolerance for the Iteration  14 :  958.3618424469025
Tolerance for the Iteration  15 :  828.6534768586473
Tolerance for the Iteration  16 :  642.5094793415824
Tolerance for the Iteration  17 :  470.5931554241213
Tolerance for the Iteration  18 :  324.011794274

In [24]:
Cosine_SSE = model3.calculate_SSE(centroids3,clusters3,data)

In [25]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25490189976.302994
Jacard SSE: 34361687572.938736
Cosine SSE : 25423847135.78622


In [26]:
cluster_labels3 = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels3

array([7, 2, 9, 6, 0, 3, 1, 5, 4, 0])

In [32]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

0.0925

In [33]:
print("Euclidean accuracy:",Accuracy_Euclidean)
print("Jacard accuracy:",Accuracy_Jaccard)
print("Cosine accuracy :",Accuracy_Cosine)

Euclidean accuracy: 0.0925
Jacard accuracy: 0.1075
Cosine accuracy : 0.0925
